In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
from fastai import *
from fastai.vision import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import torch

In [3]:
Path = '../input/gala-dataset/dataset/'

In [4]:
train_df = pd.read_csv(Path+'train.csv')
test_df = pd.read_csv(Path+'test.csv')

In [5]:
train_df.head()

,Image,Class
0,image7042.jpg,Food
1,image3327.jpg,misc
2,image10335.jpg,Attire
3,image8019.jpg,Food
4,image2128.jpg,Attire


In [6]:
def create_data(bs, Size, train_idx, val_idx):
    tfms = get_transforms(do_flip=True, flip_vert=False, max_rotate=45.0, max_zoom=1.3, max_lighting=0.5)
    
    data = (
        ImageList.from_df(df=train_df, folder='Train Images', path=Path)
        .split_by_idxs(train_idx, val_idx)
       .label_from_df(cols='Class')
       .transform(tfms,size=Size)
       .databunch(bs=bs)
       .normalize(imagenet_stats)
    )
    
    
    return data


def add_test_data(data):
    test_data = (ImageList.from_df(df=test_df,folder='Test Images',path=Path))
    data.add_test(test_data)
    return data

In [7]:
from sklearn.model_selection import StratifiedKFold
from fastai.metrics import FBeta

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fscores = []
test = []

tfms = get_transforms(do_flip=True, flip_vert=False, max_rotate=45.0, max_zoom=1.3, max_lighting=0.5)


for train_index, val_index in skf.split(train_df.index, train_df['Class']):
    data_fold = create_data(32, 128, train_index, val_index)
    data_fold = add_test_data(data_fold)
    learn = cnn_learner(data_fold, models.resnet50, metrics=[FBeta(average='weighted', beta=1)],pretrained=True,path='../working/')
    learn.fit_one_cycle(8, 1e-03)
    fscores.append(learn.recorder.metrics)
    pred, y = learn.get_preds(DatasetType.Test)
    test.append(pred)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


epoch,train_loss,valid_loss,f_beta,time
0,1.462459,0.815837,0.734362,00:32
1,1.178431,0.699872,0.762583,00:25
2,0.891614,0.576445,0.790784,00:27
3,0.726726,0.524300,0.813031,00:25
4,0.687885,0.493804,0.814981,00:27
5,0.602831,0.460582,0.833790,00:25
6,0.532641,0.449088,0.840996,00:26
7,0.502321,0.444358,0.836480,00:26


epoch,train_loss,valid_loss,f_beta,time
0,1.393049,0.711884,0.747616,00:27
1,1.142058,0.672955,0.780243,00:26
2,0.892783,0.518291,0.809997,00:26
3,0.736130,0.470113,0.833534,00:27
4,0.657842,0.451761,0.836167,00:27
5,0.608824,0.412311,0.851467,00:28
6,0.556994,0.400595,0.862942,00:26
7,0.523034,0.406834,0.861947,00:28


epoch,train_loss,valid_loss,f_beta,time
0,1.449451,0.862131,0.716956,00:26
1,1.175161,0.790733,0.763664,00:29
2,0.904521,0.602927,0.789945,00:27
3,0.751210,0.544610,0.798530,00:27
4,0.651962,0.504163,0.827285,00:26
5,0.583909,0.469521,0.829228,00:27
6,0.522718,0.473715,0.828582,00:26
7,0.490340,0.466913,0.830649,00:27


epoch,train_loss,valid_loss,f_beta,time
0,1.390649,0.839613,0.738673,00:26
1,1.176332,0.801711,0.752140,00:27
2,0.906251,0.624956,0.765385,00:25
3,0.744217,0.571590,0.784654,00:26
4,0.639641,0.522674,0.792308,00:26
5,0.574870,0.504810,0.805533,00:25
6,0.560992,0.491004,0.809648,00:27
7,0.508930,0.485836,0.818491,00:25


epoch,train_loss,valid_loss,f_beta,time
0,1.395851,0.798757,0.743221,00:27


In [8]:
final_pred = (test[0]+test[1]+test[2]+test[3]+test[4])/5

In [9]:
final_pred.size()

torch.Size([3219, 4])

In [10]:
final_pred = final_pred.argmax(dim=1)

In [11]:
sample_df = test_df.copy()
sample_df['Class'] = 0
sample_df['Class'] = final_pred
sample_df.head()

,Image,Class
0,image6245.jpg,2
1,image10409.jpg,2
2,image8692.jpg,2
3,image10517.jpg,2
4,image2580.jpg,0


In [12]:
d = {0:'Attire', 1:'Decorationandsignage', 2:'Food', 3:'misc'}

In [13]:
sample_df.replace({"Class":d}, inplace=True)
sample_df.head()

,Image,Class
0,image6245.jpg,Food
1,image10409.jpg,Food
2,image8692.jpg,Food
3,image10517.jpg,Food
4,image2580.jpg,Attire


In [14]:
sample_df.to_csv('best_cv.csv', index=False)